<div align="center">
    <h1>Heart Disease</h1>
</div>

<div>
    <h1>Introduction to Artificial Intelligence | Project 2 | Universidad del Valle</h1>
</div>

![Imagen](https://img.webmd.com/dtmcms/live/webmd/consumer_assets/site_images/articles/health_tools/did_you_know_this_could_lead_to_heart_disease_slideshow/493ss_thinkstock_rf_heart_illustration.jpg)

## Authors
- Bryan Steven Biojó     - 1629366
- Julián Andrés Castaño  - 1625743
- Juan Sebastián Saldaña - 1623447
- Juan Pablo Rendón      - 1623049

## Objective
- Apply the concept of Machine Learning (ML) to solve a **classification problem** using the methods seen in the course.

## 1. Importing libraries
As a first step, the libraries used during the development of the problem will be imported:

In [3]:
# Common libraries
import numpy as np
import pandas as pd
import seaborn as sb
import tensorflow as tf
import matplotlib.pyplot as plt
import math
import re
import os
import sys
import warnings
warnings.filterwarnings('ignore')
from matplotlib.legend_handler import HandlerLine2D
from IPython.display import SVG, display
from graphviz import Source

# Sklearn libraries
import sklearn
from sklearn import tree
from sklearn.tree import export_graphviz
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
#from sklearn.externals.six import StringIO

# Keras libraries
from keras.models import Sequential
from keras.layers.core import Dense

# Libraries just in case...
"""
# Disabling TensorFlow warning messages
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Function that captures warnings
def warn(*args, **kwargs):
    pass
"""

## 2. Loading the dataset
Next, we will load the dataset with the heart diseases which are in a CSV file. Said data was extracted directly from the **Kaggle** website (https://www.kaggle.com/ronitf/heart-disease-uci) and uploaded again to the following **GitHub** repository (https://github.com/bryansbr/heart-disease-AI:

In [4]:
# Dataset
url = "https://raw.githubusercontent.com/bryansbr/heart-disease-AI/main/heart.csv"
data = pd.read_csv(url)
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
